In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri July 22 12:17:26 2020

@author: vsharma
"""
import pyodbc as pdb
import pandas as pd
import numpy as np

#conn = pdb.connect("DRIVER={Ingres};SERVER=@10.100.13.111,tcp_ip,27712;DATABASE=udfvid;uid=actian;pwd=plug$play")
#conn = pdb.connect("dsn=iris1;uid=actian;pwd=plug$play" )
conn = pdb.connect("dsn=Vector6;uid=actian;pwd=actian" )
#conn.autocommit= True
conn.setdecoding(pdb.SQL_CHAR, encoding='utf-8')
conn.setdecoding(pdb.SQL_WCHAR, encoding='utf-8')
conn.setencoding(encoding='utf-8')
conn.setdecoding(pdb.SQL_WMETADATA, encoding='utf-8')

#conn.autocommit= True
cursor = conn.cursor()



In [2]:
iris1tbl ='''create table iris( 
        id integer,
        sepallengthcm float,
        sepalwidthcm float,
        petallengthcm float,
        petalwidthcm float,
        species varchar(20))
        with NOPARTITION;'''
conn.execute (iris1tbl)
conn.commit()


In [3]:
#Bluk load data using copy vwload
query ="COPY iris() VWLOAD FROM '/home/actian/iris/datasets_19_420_Iris.csv' with fdelim=',', insertmode ='Bulk' ,header"
cursor.execute(query)
cursor.commit()

In [4]:
sql= "select count(*) as count, species from iris group by species"

iris=pd.read_sql(sql, conn)
print(iris.shape)
iris.info(verbose=True)
iris.describe()
iris.head()

(3, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
c o u      3 non-null int64
s p e c    3 non-null object
dtypes: int64(1), object(1)
memory usage: 176.0+ bytes


,c o u,s p e c
0,50,Iris-setosa
1,50,Iris-versicolor
2,50,Iris-virginica


In [5]:
sql_case="select sepallengthcm, sepalwidthcm,petallengthcm, petalwidthcm , \
    CASE \
    WHEN species='Iris-setosa' THEN '1' \
    WHEN species='Iris-versicolor' THEN '2' \
    ELSE '3' \
    END as species \
    FROM iris" 
iris_case=pd.read_sql(sql_case, conn)
print(iris_case.shape)
iris_case.info(verbose=True)

(150, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
s e p a l l e    150 non-null float64
s e p a l w      150 non-null float64
p e t a l l e    150 non-null float64
p e t a l w      150 non-null float64
s p e c          150 non-null object
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [6]:
#Train and test the model
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


# Load iris
x_data = iris_case.iloc[:, 0:4] 
y_data = iris_case.iloc[:, 4:5] 
#y_data = iris_case[5:6]

print(x_data.head(10))
print(y_data.head(10))
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data ,test_size = 0.3, shuffle=True)


clf = LogisticRegression(random_state=0)
clf.fit(x_train, y_train)


   s e p a l l e  s e p a l w   p e t a l l e  p e t a l w 
0            5.1           3.5            1.4           0.2
1            4.9           3.0            1.4           0.2
2            4.7           3.2            1.3           0.2
3            4.6           3.1            1.5           0.2
4            5.0           3.6            1.4           0.2
5            5.4           3.9            1.7           0.4
6            4.6           3.4            1.4           0.3
7            5.0           3.4            1.5           0.2
8            4.4           2.9            1.4           0.2
9            4.9           3.1            1.5           0.1
  s p e c
0       1
1       1
2       1
3       1
4       1
5       1
6       1
7       1
8       1
9       1


d:\Users\vsharma\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\Users\vsharma\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\Users\vsharma\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
#Check the accuracy of the model

ypred_test = clf.predict(x_test)
print (ypred_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, ypred_test))

print(clf)
print(clf.coef_, clf.intercept_)



['2' '2' '2' '1' '2' '2' '3' '1' '2' '2' '2' '2' '3' '3' '2' '3' '1' '1'
 '3' '3' '1' '3' '1' '1' '3' '3' '3' '1' '3' '1' '1' '3' '2' '1' '3' '2'
 '1' '3' '3' '2' '3' '2' '1' '2' '3']
0.8888888888888888
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
[[ 0.3895888   1.34595886 -2.06997638 -0.94121797]
 [ 0.47592821 -1.57379327  0.42658526 -1.13567644]
 [-1.54892257 -1.15909567  2.15560176  2.18695775]] [ 0.25519024  0.68637376 -0.91705659]


In [8]:
# m2cgen will help in transpile the code to python, which can be converted to UDF
import m2cgen as m2c
code = m2c.export_to_python(clf)
print(code)

def score(input):
    return [((((0.25519023562092724) + ((input[0]) * (0.3895887983224221))) + ((input[1]) * (1.345958860267565))) + ((input[2]) * (-2.0699763848885673))) + ((input[3]) * (-0.9412179668992701)), ((((0.6863737639240429) + ((input[0]) * (0.47592820981827283))) + ((input[1]) * (-1.573793271545845))) + ((input[2]) * (0.42658525769464284))) + ((input[3]) * (-1.1356764428153145)), ((((-0.9170565873708041) + ((input[0]) * (-1.5489225723875584))) + ((input[1]) * (-1.1590956697735213))) + ((input[2]) * (2.155601761372578))) + ((input[3]) * (2.1869577475772353))]



In [9]:
# formatting properly for python UDF and apply the sigmoid appropriately
udf='''CREATE OR REPLACE FUNCTION iris_lr_Python(lst VARCHAR(100) not null) 
              return (int not null) 
              AS LANGUAGE PYTHON
              SOURCE='
import math
def score (lst):
    input = [float(i) for i in lst.split(",")]
    var0= [((((0.3071141523497695) + ((input[0]) * (0.45735289079282243))) + ((input[1]) * (1.563999442624823))) + ((input[2]) * (-2.4337731386698156))) + ((input[3]) * (-1.1093664333108562)), ((((1.3549694262711482) + ((input[0]) * (0.383714697433802))) + ((input[1]) * (-1.6331343435222105))) + ((input[2]) * (0.5223067330148571))) + ((input[3]) * (-1.2217422222774255)), ((((-1.5634849658960563) + ((input[0]) * (-1.9559633315117064))) + ((input[1]) * (-1.6394612788542173))) + ((input[2]) * (2.83919578446464))) + ((input[3]) * (2.777913439552842))]
    print(var0)
    index=0
    arr1=[]
    for i in range(0,len(var0)): 
     arr1.append( 1 / (1 + math.exp(-var0[i])))
    print(arr1)
    max_val=arr1[0]
    for i in range(0,len(arr1)):
           if arr1[i] > max_val:
                index=i
                max_val=arr1[i]
    return index+1
return score (lst)'
'''



cursor.execute(udf)
conn.commit()




In [10]:
#Check the test prediction with UDF
predict='''select CASE 
    WHEN species='Iris-setosa' THEN '1' 
    WHEN species='Iris-versicolor' THEN '2' 
    ELSE '3' 
    END as species,iris_lr_Python(sepallengthcm  || ',' || sepalwidthcm  || ',' || petallengthcm  || ',' || petalwidthcm ) as classify  from iris'''
predict1=pd.read_sql(predict, conn)
predict1.head()

,s p e c,c l a s 
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [11]:
# now the model transpiled to Javascript code
code = m2c.export_to_javascript(clf)
print(code)


function score(input) {
    return [((((0.25519023562092724) + ((input[0]) * (0.3895887983224221))) + ((input[1]) * (1.345958860267565))) + ((input[2]) * (-2.0699763848885673))) + ((input[3]) * (-0.9412179668992701)), ((((0.6863737639240429) + ((input[0]) * (0.47592820981827283))) + ((input[1]) * (-1.573793271545845))) + ((input[2]) * (0.42658525769464284))) + ((input[3]) * (-1.1356764428153145)), ((((-0.9170565873708041) + ((input[0]) * (-1.5489225723875584))) + ((input[1]) * (-1.1590956697735213))) + ((input[2]) * (2.155601761372578))) + ((input[3]) * (2.1869577475772353))];
}



In [12]:
# formatting properly for Javascript UDF, returning results as per maximum likelyhood
udf_JS ='''create function  iris_lr_javascript(lst VARCHAR(100) not
null) return (int not null) AS LANGUAGE JAVASCRIPT SOURCE='
function score(lst) {
    var input = lst.split(",").map(function(item) {  return
parseFloat(item); });
    nums=[((((0.3071141523497695) + ((input[0]) * (0.45735289079282243))) + ((input[1]) * (1.563999442624823))) + ((input[2]) * (-2.4337731386698156))) + ((input[3]) * (-1.1093664333108562)), ((((1.3549694262711482) + ((input[0]) * (0.383714697433802))) + ((input[1]) * (-1.6331343435222105))) + ((input[2]) * (0.5223067330148571))) + ((input[3]) * (-1.2217422222774255)), ((((-1.5634849658960563) + ((input[0]) * (-1.9559633315117064))) + ((input[1]) * (-1.6394612788542173))) + ((input[2]) * (2.83919578446464))) + ((input[3]) * (2.777913439552842))]
var index = 0;
        for (var i = 0; i < nums.length; i++) {
            index = nums[i] > nums[index] ? i : index;
        }
        return index+1;
}
return score(lst)'
'''
cursor.execute(udf_JS)
conn.commit()

In [13]:
#Predicting with Javascript UDF
predictJS='''select CASE 
    WHEN species='Iris-setosa' THEN '1' 
    WHEN species='Iris-versicolor' THEN '2' 
    ELSE '3' 
    END as species,iris_lr_javascript(sepallengthcm  || ',' || sepalwidthcm  || ',' || petallengthcm  || ',' || petalwidthcm ) as classify  from iris'''
predict2=pd.read_sql(predictJS, conn)
predict2.head()

,s p e c,c l a s 
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1


In [14]:
#Make the JSON with intercept and coeff of the model
import json  
model_param = {}
model_param['coef'] = clf.coef_.tolist() 
model_param['intercept'] =clf.intercept_.tolist()
b = json.dumps(model_param) 
print(b)

{"coef": [[0.3895887983224221, 1.345958860267565, -2.0699763848885673, -0.9412179668992701], [0.47592820981827283, -1.573793271545845, 0.42658525769464284, -1.1356764428153145], [-1.5489225723875584, -1.1590956697735213, 2.155601761372578, 2.1869577475772353]], "intercept": [0.25519023562092724, 0.6863737639240429, -0.9170565873708041]}


In [15]:
#Create JSON Data UDF and again returning results as per maximum likelihood

jsonstr='''create function  iris_lr_json(lst VARCHAR(100) not null)
return (int not null) AS LANGUAGE python SOURCE='
import json
def score1 (lst):
  input = [float(i) for i in lst.split(",")]
  json_data = "{\\"coef\\": [[0.4156346233598447, 1.6410466066307934, \
-2.4486206310542746, -1.0775840518519904], [0.7334652955571552, \
-2.0527321763924102, 0.4049181379167901, -1.4276343377888174], \
[-1.8886197150361799, -1.701005052157171, 2.803528886728411, \
2.6749639949986195]], \\"intercept\\": [0.2834483510261381, \
1.271492823301712, -1.403879236024429]}"
  dictjson = json.loads(json_data)
  # raise Exception("it is %s" % str(dictjson))
  var0=[dictjson["intercept"][0]+ input[0]*dictjson["coef"][0][0] + 
input[1]*dictjson["coef"][0][1]+input[2]*dictjson["coef"][0][2] + 
input[3]*dictjson["coef"][0][3], dictjson["intercept"][1]+
input[0]*dictjson["coef"][1][0] +input[1]*dictjson["coef"][1][1]+
input[2]*dictjson["coef"][1][2] + input[3]*dictjson["coef"][1][3],
dictjson["intercept"][2]+ input[0]*dictjson["coef"][2][0] +
input[1]*dictjson["coef"][2][1]+input[2]*dictjson["coef"][2][2] +
input[3]*dictjson["coef"][2][3]]
  #print(var0)
  index=0
  max_val=var0[0]
  for i in range(0,len(var0)):
    if var0[i] > max_val:
      index=i
      max_val=var0[i]
  return index+1
return score (lst)'
'''

cursor.execute(jsonstr)
conn.commit()


In [16]:
#Predict with JSON UDF
predictJson='''select CASE 
    WHEN species='Iris-setosa' THEN '1' 
    WHEN species='Iris-versicolor' THEN '2' 
    ELSE '3' 
    END as species,iris_lr_javascript(sepallengthcm  || ',' || sepalwidthcm  || ',' || petallengthcm  || ',' || petalwidthcm ) as classify  from iris'''
predict3=pd.read_sql(predictJson, conn)
predict3.head()

,s p e c,c l a s 
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
